In [1]:
from PIL import Image
import numpy as np

In [6]:
# Declaring the image path
img_pth = "test.png"

# Converting the image to a grayscale numpy array
img = np.array(Image.open(img_pth).convert("L"), dtype=np.uint8)
[rows, cols] = img.shape

print(f"No of rows in the image: {rows}")
print(f"No of columns in the image: {cols}")

No of rows in the image: 265
No of columns in the image: 265


In [8]:
size = rows*cols

# Storing the image pix to a 1d array for easier implementation of segmentation
img_pix = np.resize(img, (size))

print(f"The size of the img_pix array: {len(img_pix)}")

The size of the img_pix array: 70225


In [14]:
# conversion from img pix position to hash value of array
def pos_to_hash(row, col):
    hash_value = row*rows + col
    return hash_value

# conversion from hash value of array to img pix position
def hash_to_pix(hash_value):
    row = hash_value%rows
    col = hash_value - row*rows
    return (row, col)

In [12]:
class DSU:
    def __init__(self, N):
        self.nodes = [i for i in range(N)]
    
    def find(self, x):
        if self.nodes[x] != x:
            self.nodes[x] = self.find(self.nodes[x])
        return self.nodes[x]
    
    def union(self, x, y):
        rootA, rootB = self.find(x), self.find(y)
        if rootA != rootB:
            self.nodes[rootA] = rootB

In [13]:
segment = DSU(size)

In [20]:
edges = []

# The weight function w in for each edge
def w(p1, p2):
    return abs(int(img_pix[p1]) - int(img_pix[p2]))

# Generation edges by iterating one by one over all nodes
for i in range(rows):
    for j in range(cols):

        no = pos_to_hash(i, j)    # the center node
        nr = pos_to_hash(i, j+1)  # right neighbour of the center node
        nb = pos_to_hash(i+1, j)  # bottom neighbour of the center node

        if j != cols-1:
            edges.append([w(no, nr), [no, nr]])
        if i != rows-1:
            edges.append([w(no, nb), [no, nb]])

# Sorting the edges based on their weights
edges.sort(key=lambda x: x[0])

In [79]:


for edge in edges:
    weight = edge[0]
    n1 = edge[1][0]
    n2 = edge[1][1]
    c1 = segment.find(n1)
    c2 = segment.find(n2)
    mint = min(segment.Int(c1) + t(segment.size(c1)), segment.Int(c2) + t(segment.size(c2)))

0 0 1
0 0 265
0 1 2
0 1 266
0 2 3
0 2 267
0 3 268
0 4 5
0 4 269
0 5 6
0 5 270
0 6 7
0 6 271
0 7 272
0 8 9
0 8 273
0 9 10
0 9 274
0 10 11
0 10 275
0 11 276
0 12 13
0 12 277
0 13 14
0 13 278
0 14 15
0 14 279
0 15 280
0 16 17
0 16 281
0 17 18
0 17 282
0 18 19
0 18 283
0 19 284
0 20 21
0 20 285
0 21 22
0 21 286
0 22 23
0 22 287
0 23 288
0 24 25
0 24 289
0 25 26
0 25 290
0 26 27
0 26 291
0 27 28
0 27 292
0 28 293
0 29 30
0 29 294
0 30 31
0 30 295
0 31 32
0 31 296
0 32 297
0 33 34
0 33 298
0 34 35
0 34 299
0 35 36
0 35 300
0 36 301
0 37 38
0 37 302
0 38 39
0 38 303
0 39 40
0 39 304
0 40 305
0 41 42
0 41 306
0 42 43
0 42 307
0 43 44
0 43 308
0 44 309
0 45 46
0 45 310
0 46 47
0 46 311
0 47 48
0 47 312
0 48 313
0 49 50
0 49 314
0 50 51
0 50 315
0 51 52
0 51 316
0 52 53
0 52 317
0 53 318
0 54 55
0 54 319
0 55 56
0 55 320
0 56 57
0 56 321
0 57 322
0 58 59
0 58 323
0 59 60
0 59 324
0 60 61
0 60 325
0 61 326
0 62 63
0 62 327
0 63 64
0 63 328
0 64 65
0 64 329
0 65 330
0 66 67
0 66 331
0 67 68
0 67 3